In [36]:
#importações das bibliotecas
import pandas as pd
import requests

## Campeonato brasileiro

In [ ]:
seriaA2024 = requests.get('https://pt.wikipedia.org/wiki/Campeonato_Brasileiro_de_Futebol_de_2024_-_S%C3%A9rie_A')

# Informações do campeonato de 2024
tabelas2024 = pd.read_html(seriaA2024.text)
tabela_classificação2024 = tabelas2024[6]
tabela_jogos2024 = tabelas2024[7]

In [38]:
#criando uma lista com os nomes dos times
nomes_times = list(tabela_jogos2024["Casa \ Fora"])
#criando uma lista com as siglas dos times
sigla_times = list(tabela_jogos2024.columns)
sigla_times.pop(0)

#criando um dicionário com a sigla dos times
apelidos_times = dict(zip(sigla_times, nomes_times))


In [ ]:
# Ajustar a tabela de jogos
tabela_jogos2024_ajustada = tabela_jogos2024.set_index("Casa \ Fora")
# Desempilhar a tabela
tabela_jogos2024_ajustada = tabela_jogos2024_ajustada.unstack().reset_index()
# Renomear as colunas
tabela_jogos2024_ajustada = tabela_jogos2024_ajustada.rename(columns={"level_0": "Fora", "Casa \ Fora": "Casa", 0: "Placar"})

# Função para ajustar o nome dos times
def ajustar_apelido(linha):
    time = linha["Fora"]
    return apelidos_times[time]

# Ajustar o nome dos times aplicando a função
tabela_jogos2024_ajustada["Fora"] = tabela_jogos2024_ajustada.apply(ajustar_apelido, axis=1)
# Remover jogos entre o mesmo time
tabela_jogos2024_ajustada = tabela_jogos2024_ajustada[tabela_jogos2024_ajustada["Fora"] != tabela_jogos2024_ajustada["Casa"]]


In [74]:
tabela_jogos2024_ajustada["Placar"] = tabela_jogos2024_ajustada["Placar"].fillna("A jogar")

jogos_realizados = tabela_jogos2024_ajustada[tabela_jogos2024_ajustada["Placar"].str.contains("–")]
jogos_faltantes = tabela_jogos2024_ajustada[~tabela_jogos2024_ajustada["Placar"].str.contains("–")]
jogos_faltantes = jogos_faltantes.drop(columns=["Placar"])


In [66]:
# Separando a coluna Placar em gols_casa e gols_fora
jogos_realizados["gols_casa"] = jogos_realizados["Placar"].str.split("–").str[0].astype(int)
jogos_realizados["gols_fora"] = jogos_realizados["Placar"].str.split("–").str[1].astype(int)
jogos_realizados = jogos_realizados.drop(columns=["Placar"])

# Ajustar o tipo das colunas
jogos_realizados["gols_casa"] = jogos_realizados["gols_casa"].astype(int)
jogos_realizados["gols_fora"] = jogos_realizados["gols_fora"].astype(int)
display(jogos_realizados)

C:\Users\Thiago\AppData\Local\Temp\ipykernel_13464\1880760640.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jogos_realizados["gols_casa"] = jogos_realizados["Placar"].str.split("–").str[0].astype(int)
C:\Users\Thiago\AppData\Local\Temp\ipykernel_13464\1880760640.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  jogos_realizados["gols_fora"] = jogos_realizados["Placar"].str.split("–").str[1].astype(int)


,Fora,Casa,gols_casa,gols_fora
11,Athletico Paranaense,Fortaleza,1,0
12,Athletico Paranaense,Grêmio,2,0
14,Athletico Paranaense,Juventude,1,1
15,Athletico Paranaense,Palmeiras,0,2
24,Atlético Goianiense,Botafogo,1,0
...,...,...,...,...
367,Vitória,Cruzeiro,3,1
379,Vitória,Vasco da Gama,2,1
380,Vasco da Gama,Athletico Paranaense,1,0
390,Vasco da Gama,Fluminense,2,1


In [73]:
# Media de gols feitos dentro de casa de um time
# Essa Tabela diz a media de gols que o time fez em casa e a media de gols que o time sofreu dentro de casa
media_gols_casa = jogos_realizados.groupby("Casa").mean(numeric_only=True)
media_gols_casa = media_gols_casa.rename(columns={"gols_casa": "gols feitos", "gols_fora": "gols sofridos"})
display(media_gols_casa)
# Media de gols feitos e sofridos fora de casa de um time
media_gols_fora = jogos_realizados.groupby("Fora").mean(numeric_only=True)
media_gols_fora = media_gols_fora.rename(columns={"gols_casa": "gols sofridos", "gols_fora": "gols feito"})
display(media_gols_fora)


,gols feitos,gols sofridos
Casa,,
Athletico Paranaense,2.000000,0.000000
Atlético Goianiense,0.333333,2.000000
Atlético Mineiro,1.666667,0.666667
Bahia,1.333333,0.333333
Botafogo,2.333333,1.000000
Corinthians,0.750000,0.250000
Criciúma,1.000000,1.500000
Cruzeiro,3.000000,1.500000
Cuiabá,0.000000,2.000000


,gols sofridos,gols feito
Fora,,
Athletico Paranaense,1.000000,0.750000
Atlético Goianiense,0.666667,1.000000
Atlético Mineiro,0.666667,1.666667
Bahia,1.500000,1.500000
Botafogo,1.000000,1.500000
Corinthians,1.666667,0.000000
Criciúma,0.500000,2.500000
Cruzeiro,1.500000,0.500000
Cuiabá,2.500000,0.000000
